In [1]:
import os
os.getcwd()

'/work/TALC/enel645_2025w'

In [2]:
import os
os.listdir()

['garbage_data',
 'local_dataset',
 'transfer_learning',
 'slurm-35541.out',
 'best_model.pth',
 'CombinedModel.py',
 '.git',
 'cm1.slurm',
 'cm1.py',
 'jupyter_env',
 'Assignment2.slurm',
 '.ipynb_checkpoints',
 'test.ipynb']

In [3]:
import os
os.chdir('/work/TALC/enel645_2025w/garbage_data')

In [5]:
import os
os.listdir()

['CVPR_2024_dataset_Train',
 'CVPR_2024_dataset_Test',
 'CVPR_2024_dataset_Val',
 'Downloads',
 '.ipynb_checkpoints',
 'Data',
 'import os.py',
 'source',
 'garbage_data',
 'train.py',
 'Sampled_Train',
 'local_garbage_data',
 'Sampled_Val',
 'Sampled_Test',
 'garbage_data.tar.gz',
 'my_venv']

In [7]:
!pip install torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor()           # Convert to Tensor
])

# Load dataset
dataset_path = "/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Train"
train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Check a sample
image, label = train_dataset[0]
print(image.shape, label)

     |████████████████████████████████| 23.3 MB 2.7 MB/s            
     |████████████████████████████████| 881.9 MB 9.8 kB/s             
     |████████████████████████████████| 3.1 MB 50.4 MB/s            
     |████████████████████████████████| 14.8 MB 34.1 MB/s            
torch.Size([3, 224, 224]) 0


In [9]:
!pip install scikit-learn

     |████████████████████████████████| 22.2 MB 2.8 MB/s            
     |████████████████████████████████| 25.9 MB 49.4 MB/s            
     |████████████████████████████████| 309 kB 50.1 MB/s            


In [11]:
!pip install pandas

     |████████████████████████████████| 9.5 MB 3.0 MB/s            


In [12]:
# --- Imports ---
import os
#import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np
from PIL import Image # To handle images

# --- Define Dataset Class ---
class GarbageDataset(Dataset):
    def __init__(self, data_dir, split='train', transform=None, text_transform=None):
        """
        Args:
            data_dir (string): Directory with all the data (train, val, test folders).
            split (string): 'train', 'val', or 'test'.
            transform (callable, optional): Optional transform to be applied on image.
            text_transform (callable, optional): Optional transform to be applied on text data (if applicable).
        """
        self.data_dir = os.path.join(data_dir, f'CVPR_2024_dataset_{split.capitalize()}') # Assuming folder structure
        self.split = split
        self.transform = transform
        self.text_transform = text_transform
        self.image_dir = os.path.join(self.data_dir, 'images')
        self.text_file = os.path.join(self.data_dir, f'{split}_labels.txt') # Assuming labels are in a text file

        self.image_filenames = []
        self.labels = []
        self.texts = [] # Placeholder for text data if available

        # --- Load Data ---
        with open(self.text_file, 'r') as f:
            for line in f:
                filename, label = line.strip().split() # Assuming format "filename label" in txt file
                self.image_filenames.append(filename)
                self.labels.append(int(label)) # Assuming labels are integers. Adjust if needed.
                # --- Load Textual Data (Example - Adapt based on your actual text data format) ---
                # For example, if you have text descriptions in separate files named like image filenames
                text_filepath = os.path.join(self.data_dir, 'texts', filename.replace('.jpg', '.txt')) # Example path
                if os.path.exists(text_filepath):
                    with open(text_filepath, 'r', encoding='utf-8') as text_f: # Handle encoding if necessary
                        text_content = text_f.read().strip()
                        self.texts.append(text_content)
                else:
                    self.texts.append("") # Or handle missing text as needed


    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        image = Image.open(img_path).convert('RGB') # Ensure RGB for consistency
        label = self.labels[idx]
        text = self.texts[idx]

        if self.transform:
            image = self.transform(image)
        # if self.text_transform: # Apply text transform if needed (e.g., tokenization, embedding)
        #     text = self.text_transform(text) # Placeholder - Implement text transform if necessary

        return image, text, label # Return image, text, and label


# --- Define Data Transformations ---
image_transform = transforms.Compose([
    transforms.Resize((256, 256)), # Or other appropriate size
    transforms.RandomHorizontalFlip(), # Data augmentation - adjust as needed
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet normalization - adjust if needed
])

# --- Text Transformation (Placeholder - Adapt based on your text data and model) ---
# Example using simple tokenization (you might need more advanced techniques like BERT, etc.)
# def simple_tokenizer(text):
#     return text.split()
# text_transform = simple_tokenizer # Example, replace with your actual text processing


# --- Instantiate Datasets ---
data_dir = '/work/TALC/enel645_2025w/garbage_data' # Replace with your actual data directory

train_dataset = GarbageDataset(data_dir, split='train', transform=image_transform) #, text_transform=text_transform) # Add text_transform if you define one
val_dataset = GarbageDataset(data_dir, split='val', transform=image_transform) #, text_transform=text_transform)
test_dataset = GarbageDataset(data_dir, split='test', transform=image_transform) #, text_transform=text_transform)


# --- Data Loaders ---
batch_size = 32 # Adjust batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4) # Adjust num_workers
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


# --- Define Model Architecture ---
class GarbageClassifier(nn.Module):
    def __init__(self, num_classes):
        super(GarbageClassifier, self).__init__()
        # --- Image Feature Extractor (Example - ResNet18) ---
        self.image_encoder = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True) # Or other CNN
        # Remove last layers for feature extraction
        self.image_encoder = nn.Sequential(*list(self.image_encoder.children())[:-2]) # Remove avgpool and fc

        # --- Text Feature Extractor (Placeholder - Example: Simple Embedding and RNN) ---
        # You'll need to adapt this based on your text data and desired complexity
        # self.embedding = nn.Embedding(vocab_size, embedding_dim) # Define vocab_size and embedding_dim
        # self.text_rnn = nn.LSTM(embedding_dim, hidden_size, batch_first=True) # Define hidden_size

        # --- Fusion Layer (Example - Concatenation and Linear layer) ---
        # self.fusion_layer = nn.Linear(image_feature_size + text_feature_size, fusion_output_size) # Define sizes
        self.fusion_layer = nn.Linear(512, 256) # Example ResNet18 feature size is 512, adjust if needed, remove text part for now

        # --- Classification Head ---
        self.classifier = nn.Linear(256, num_classes) # From fusion output to number of classes


    def forward(self, images, texts): # Expects image and text batches
        # --- Image Feature Extraction ---
        image_features = self.image_encoder(images) # [batch_size, C, H, W]
        image_features = torch.mean(image_features, dim=(-2, -1)) # Global Average Pooling [batch_size, C]


        # --- Text Feature Extraction (Placeholder - Adapt based on your text model) ---
        # text_embedded = self.embedding(texts) # [batch_size, seq_len, embedding_dim]
        # text_features, _ = self.text_rnn(text_embedded) # [batch_size, seq_len, hidden_size]
        # text_features = text_features[:, -1, 🙂 # Take last time step's output [batch_size, hidden_size] # Or use attention, etc.

        # --- Feature Fusion (Example - Concatenation) ---
        # combined_features = torch.cat((image_features, text_features), dim=1) # Concatenate image and text features
        combined_features = image_features # Removing text for now

        fused_features = self.fusion_layer(combined_features)
        fused_features = torch.relu(fused_features) # Activation function

        # --- Classification ---
        output = self.classifier(fused_features)
        return output



# --- Instantiate Model, Loss, Optimizer ---
num_classes = 6 # Example number of garbage classes - adjust based on your dataset
model = GarbageClassifier(num_classes=num_classes)

criterion = nn.CrossEntropyLoss() # Common loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001) # Common optimizer, adjust learning rate as needed

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Use GPU if available
model.to(device)


# --- Training Loop ---
num_epochs = 10 # Adjust number of epochs
for epoch in range(num_epochs):
    model.train() # Set model to training mode
    train_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, texts, labels in train_loader:
        images = images.to(device)
        # texts = texts.to(device) # If you process text as tensors - adapt text processing
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images, texts) # Pass both image and text to the model
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    model.eval() # Set model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad(): # Disable gradient calculation during validation
        for images, texts, labels in val_loader:
            images = images.to(device)
            # texts = texts.to(device) # If you process text as tensors
            labels = labels.to(device)

            outputs = model(images, texts) # Pass both image and text
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader):.4f}, Train Acc: {(correct_train/total_train)*100:.2f}%, Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {(correct_val/total_val)*100:.2f}%')


# --- Evaluation on Test Set ---
model.eval() # Ensure model is in evaluation mode
test_loss = 0.0
correct_test = 0
total_test = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, texts, labels in test_loader:
        images = images.to(device)
        # texts = texts.to(device) # If you process text as tensors
        labels = labels.to(device)

        outputs = model(images, texts)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu().numpy()) # Store predictions for metrics
        all_labels.extend(labels.cpu().numpy()) # Store true labels for metrics


print(f'Test Loss: {test_loss/len(test_loader):.4f}, Test Acc: {(correct_test/total_test)*100:.2f}%')

# --- Classification Report and Confusion Matrix ---
print("\nClassification Report:")
print(classification_report(all_labels, all_preds))

print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


# --- Save Predictions/Incorrect Classifications (Example - adapt as needed) ---
incorrect_indices = np.where(np.array(all_preds) != np.array(all_labels))[0]
incorrect_filenames = [test_dataset.image_filenames[i] for i in incorrect_indices]
incorrect_predictions = [all_preds[i] for i in incorrect_indices]
incorrect_true_labels = [all_labels[i] for i in incorrect_indices]

incorrect_df = pd.DataFrame({
    'filename': incorrect_filenames,
    'predicted_label': incorrect_predictions,
    'true_label': incorrect_true_labels
})

print("\nExamples of Incorrect Classifications:")
print(incorrect_df.head()) # Print first few incorrect examples

FileNotFoundError: [Errno 2] No such file or directory: '/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Train/train_labels.txt'

In [13]:
import os

data_root = '/work/TALC/enel645_2025w/garbage_data'

print("Contents of garbage_data directory:")
print(os.listdir(data_root)) # List the folders inside garbage_data

train_dir = os.path.join(data_root, 'CVPR_2024_dataset_Train') # Construct the train directory path
print("\nContents of CVPR_2024_dataset_Train directory:")
if os.path.exists(train_dir): # Check if the directory exists before listing
    print(os.listdir(train_dir))
else:
    print(f"Directory '{train_dir}' does not exist.")


Contents of garbage_data directory:
['CVPR_2024_dataset_Train', 'CVPR_2024_dataset_Test', 'CVPR_2024_dataset_Val', 'Downloads', '.ipynb_checkpoints', 'Data', 'import os.py', 'source', 'garbage_data', 'train.py', 'Sampled_Train', 'local_garbage_data', 'Sampled_Val', 'Sampled_Test', 'garbage_data.tar.gz', 'my_venv']

Contents of CVPR_2024_dataset_Train directory:
['Blue', 'TTR', 'Green', 'Black']


In [15]:
class GarbageDataset(Dataset):
    def __init__(self, data_dir, split='train', transform=None, text_transform=None):
        # ... (rest of the init) ...

        if split == 'train':
            split_folder = 'train_data'
        elif split == 'val':
            split_folder = 'val_data'
        elif split == 'test':
            split_folder = 'test_data'
        else:
            raise ValueError("Invalid split. Choose 'train', 'val', or 'test'.")

        self.data_dir = os.path.join(data_dir, split_folder) # Use split_folder
        self.split = split
        self.transform = transform
        self.text_transform = text_transform
        self.image_dir = os.path.join(self.data_dir, 'images')
        self.text_file = os.path.join(self.data_dir, 'labels.csv') # Filename is labels.csv now

        # ... (rest of the init, adjust label loading for CSV if needed) ...

In [16]:
self.data_dir = os.path.join(data_dir, f'CVPR_2024_dataset_{split.capitalize()}')

NameError: name 'split' is not defined

In [17]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os

In [20]:
import os
os.listdir()

['CVPR_2024_dataset_Train',
 'CVPR_2024_dataset_Test',
 'CVPR_2024_dataset_Val',
 'Downloads',
 '.ipynb_checkpoints',
 'Data',
 'import os.py',
 'source',
 'garbage_data',
 'train.py',
 'Sampled_Train',
 'local_garbage_data',
 'Sampled_Val',
 'Sampled_Test',
 'garbage_data.tar.gz',
 'my_venv']

In [18]:
class GarbageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        text = self.data.iloc[idx, 1]  # Assuming the second column is textual data
        label = int(self.data.iloc[idx, 2])  # Assuming the third column is the label

        if self.transform:
            image = self.transform(image)

        return image, text, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = GarbageDataset("train.csv", "train_images", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [36]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load datasets
train_dataset = datasets.ImageFolder(root="/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Train", transform=transform)
val_dataset = datasets.ImageFolder(root="/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Val", transform=transform)
test_dataset = datasets.ImageFolder(root="/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Test", transform=transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check class mapping
print(train_dataset.class_to_idx)

{'Black': 0, 'Blue': 1, 'Green': 2, 'TTR': 3}


In [37]:
import torch.nn as nn
import torchvision.models as models

class GarbageClassifier(nn.Module):
    def __init__(self, num_classes=6):  # Adjust `num_classes` based on your dataset
        super(GarbageClassifier, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, num_classes)

    def forward(self, image):
        return self.cnn(image)


In [ ]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GarbageClassifier(num_classes=len(train_dataset.classes)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, val_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct, total = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        print(f"Epoch {epoch+1}: Loss={running_loss / len(train_loader)}, Accuracy={train_acc:.2f}")

train_model(model, train_loader, val_loader)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/mdtahmid.jamil/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%
